In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import tflearn
from sklearn.model_selection import train_test_split
import re
tf.__version__

C:\Users\Abhishek\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


curses is not supported on this machine (please install/reinstall curses for an optimal experience)


'1.12.0'

In [2]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [3]:
df_clothing=pd.read_json("Documents/amazon_reviews/Clothing_Shoes_and_Jewelry_5.json",lines=True)
df_sports=pd.read_json("Documents/amazon_reviews/Sports_and_Outdoors_5.json",lines=True)
df=pd.concat([df_clothing,df_sports])

In [4]:
df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0000031887,"[0, 0]",5,This is a great tutu and at a really great pri...,"02 12, 2011",A1KLRMWW2FWPL4,"Amazon Customer ""cameramom""",Great tutu- not cheaply made,1297468800
1,0000031887,"[0, 0]",5,I bought this for my 4 yr old daughter for dan...,"01 19, 2013",A2G5TCU2WDFZ65,Amazon Customer,Very Cute!!,1358553600
2,0000031887,"[0, 0]",5,What can I say... my daughters have it in oran...,"01 4, 2013",A1RLQXYNCMWRWN,Carola,I have buy more than one,1357257600
3,0000031887,"[0, 0]",5,"We bought several tutus at once, and they are ...","04 27, 2014",A8U3FAMSJVHS5,Caromcg,"Adorable, Sturdy",1398556800
4,0000031887,"[0, 0]",5,Thank you Halo Heaven great product for Little...,"03 15, 2014",A3GEOILWLK86XM,CJ,Grammy's Angels Love it,1394841600


In [6]:
text=df["reviewText"].values
del df

In [7]:
text=[clean_str(sent) for sent in text]

In [11]:
max_sent_length=100
vocab_processor=tflearn.data_utils.VocabularyProcessor(max_sent_length)
data=list(vocab_processor.fit_transform(text))

Instructions for updating:
Please use tensorflow/transform or tf.data.


In [13]:
vocab_size=len(vocab_processor.vocabulary_)
vocab_size

168948

In [14]:
data=[i.tolist() for i in data]

for lst in data:
    try:
        while lst[-1]==0:
            lst.pop()
    except:
        pass
data = filter(None,data)
data = np.array(list(data))

In [20]:
print(data[0:2])
print(data.shape)

[list([1, 2, 3, 4, 5, 6, 7, 3, 8, 4, 9, 10, 11, 12, 13, 14, 7, 15, 16, 17, 18, 19, 20, 21, 22, 6, 23, 24, 25, 26, 5, 27, 2, 12, 28, 29, 3])
 list([19, 30, 1, 31, 32, 33, 34, 35, 36, 31, 37, 38, 39, 40, 10, 41, 31, 42, 43, 44, 6, 42, 45, 46, 10, 47, 48, 19, 30, 1, 49, 50, 51, 3, 52, 53, 54, 55, 56, 6, 47, 57, 42, 58, 59, 60, 4, 9, 47, 61, 62, 63, 64, 65, 66, 67, 50, 31, 68, 69, 70, 71])]
(574867,)


In [22]:
window =2
pivot_words = []
target_words = []

for d in range(data.shape[0]):
    pivot_idx = data[d][window:-window]
    
    for i in range(len(pivot_idx)):
        pivot=pivot_idx[i]
        targets=np.array([])
        neg_target=data[d][i:window+i]
        pos_target=data[d][i+window+1:i+window+window+1]
        targets=np.append(targets,[neg_target,pos_target]).flatten().tolist()
        
        for c in range(window*2):
            pivot_words.append(pivot)
            target_words.append(targets[c])
        

In [37]:
pivot_words[0:8]

[3, 3, 3, 3, 4, 4, 4, 4]

In [38]:
(target_words[0:8])

[1.0, 2.0, 4.0, 5.0, 2.0, 3.0, 5.0, 6.0]